Here I am creating a spark session object

In [ ]:

import findspark
findspark.init()


from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F


sc = SparkContext(appName="MyApp2")
spark = (
    SparkSession.builder 
    .master('local[*]')
    .config("spark.executor.memory", "4g")
    .config("spark.driver.memory", "2g")
    .getOrCreate()
)


I have extracted a common methods to clean up the datasets

split_by_symbol - this is for split the catefories by specific symbol (, or & in our cases)

lower_and_trim and add_prefix_to_col are iterating throught all columns

clean_phone - remove all symbols except a digits from a column

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql import DataFrame

def split_by_symbol(df: DataFrame, sym: str, column: str) -> DataFrame:
    """
    Splits the values in the specified column of a DataFrame by a given symbol and explodes them into separate rows.
    
    :param df: Input DataFrame
    :param sym: Symbol to split by
    :param column: Column in which to look for the symbol
    :return: DataFrame with values in the specified column split by the symbol
    """
    df_no = df.filter(~F.col(column).contains(sym) | F.col(column).isNull())
    df_yes = df.filter(F.col(column).contains(sym))
    res = df_yes.withColumn(
        column, 
        F.explode(
            F.split(df_yes[column], sym)
        )
    )
    return res.distinct().unionByName(df_no)

def lower_and_trim(df: DataFrame) -> DataFrame:
    """
    Converts all the column values of a DataFrame to lowercase and trims any spaces.
    
    :param df: Input DataFrame
    :return: DataFrame with all values in lowercase and trimmed
    """
    return df.select([F.trim(F.lower(F.col(column))).alias(column) for column in df.columns])

def add_prefix_to_cols(df: DataFrame, prefix: str) -> DataFrame:
    """
    Adds a prefix to all the column names of a DataFrame.
    
    :param df: Input DataFrame
    :param prefix: Prefix string to be added
    :return: DataFrame with the prefixed column names
    """
    return df.select([F.col(column).alias(f"{prefix}_{column}") for column in df.columns])

def clean_phone(df: DataFrame, column: str) -> DataFrame:
    """
    Cleans phone number data by removing all non-numeric characters from a specified column in a DataFrame.
    
    :param df: Input DataFrame
    :param column: Column containing phone numbers to be cleaned
    :return: DataFrame with cleaned phone numbers
    """
    return df.withColumn(column, F.regexp_replace(column, "[^0-9]", ""))


Here I'm reading the datasets and PySpark can easilly read the facebook dataset even with the errors in csv markup

Pandas couldn't do that

In [ ]:
fb_df = spark.read.csv("data/facebook_dataset.csv", header=True)
gg_df = spark.read.csv("data/google_dataset.csv", header=True)
wb_df = spark.read.csv("data/website_dataset.csv", header=True, sep=";")

In [ ]:
fb_df.show(20, truncate=False)
gg_df.show(20, truncate=False)
wb_df.show(20, truncate=False)

Here I'm selecting only needed columns and rename similar through all datasets


In [ ]:
# Restructure and rename columns 

fb_df = fb_df.select(
    "domain",
    "address",
    "categories",
    "city",
    "country_code",
    "country_name",
    "name",
    "phone",
    "region_code",
    "region_name",
    "zip_code",
)



gg_df = gg_df.select(
    "domain",
    "address",
    "category",
    "city",
    "country_code",
    "country_name",
    "name",
    "phone",
    "region_code",
    "region_name",
    "zip_code",
    "raw_address",
)



wb_df = wb_df.select(
    F.col("root_domain").alias("domain"),
    F.col("s_category").alias("category"),
    F.col("main_city").alias("city"),
    F.col("legal_name").alias("name"),
    F.col("main_region").alias("region_name"),
    F.col("phone"),
    F.col("site_name"),
)




And now we start the cleaning. I have decided to denormalise data and split categories.

So we have increased a dataset size from 72077 to 145232

In [ ]:
#Clean facebook

from pyspark.sql.functions import explode, split

#Split categories by |

fb_df_null_cat = fb_df.filter(F.col("categories").isNull()).withColumnRenamed("categories", "category")
fb_df_cat = fb_df.filter(F.col("categories").isNotNull())

fb_df_exploded = fb_df_cat.withColumn(
    "category", 
    explode(
        split(fb_df["categories"], "\|")
    )
).drop("categories")


print(fb_df.count())
print(fb_df_exploded.distinct().unionByName(fb_df_null_cat).count())

#Split category by &

fb_df_exploded_amp = split_by_symbol(fb_df_exploded, "&", "category")

print(fb_df_exploded_amp.count())

#Split category by ,

fb_df_exploded_amp = split_by_symbol(fb_df_exploded_amp, ",", "category")

print(fb_df_exploded_amp.count())

#Clean phone numbers

fb_df_exploded_amp_phone = clean_phone(fb_df_exploded_amp, "phone")

#Lower and trim all columns
#Rename cols

fb_df_clean = add_prefix_to_cols(lower_and_trim(fb_df_exploded_amp_phone), "fb")

fb_df_clean.show(10, truncate=False)

So we have increased a dataset size from 356520 to 580891

In [ ]:
#Clean google


#Split category by &

print(gg_df.count())

gg_df_amp_exploded = split_by_symbol(gg_df, "&", "category")

print(gg_df_amp_exploded.count())

#Split category by ,
gg_df_amp_exploded = split_by_symbol(gg_df_amp_exploded, ",", "category")

print(gg_df_amp_exploded.count())

#Clean phone numbers

gg_df_amp_exploded_phone = clean_phone(gg_df_amp_exploded, "phone")

#Lower and trim all columns
#Rename cols

gg_df_clean = add_prefix_to_cols(lower_and_trim(gg_df_amp_exploded_phone), "gg")

gg_df_clean.show(20, truncate=False)

In [ ]:
#Clean website

#Split category by &

print(wb_df.count())

wb_df_amp_exploded = split_by_symbol(wb_df, "&", "category")

print(wb_df_amp_exploded.count())

#Split category by ,
wb_df_amp_exploded = split_by_symbol(wb_df_amp_exploded, ",", "category")

print(wb_df_amp_exploded.count())

#Clean phone numbers

wb_df_amp_exploded_phone = clean_phone(wb_df_amp_exploded, "phone")

#Lower and trim all columns
#Rename cols
wb_df_clean = add_prefix_to_cols(lower_and_trim(wb_df_amp_exploded_phone), "wb")

wb_df_clean.show(20, truncate=False)

In addition I'm also remove all rows where name is null  

... rows was filtered

In [ ]:
print("With null names ", fb_df_clean.count())
print("With null names ", gg_df_clean.count())
print("With null names ", wb_df_clean.count())

fb_df_clean_ = fb_df_clean.filter(F.col("fb_name").isNotNull())
gg_df_clean_ = gg_df_clean.filter(F.col("gg_name").isNotNull())
wb_df_clean_ = wb_df_clean.filter(F.col("wb_name").isNotNull())

print("Without null names ", fb_df_clean_.count())
print("Without null names ", gg_df_clean_.count())
print("Without null names ", wb_df_clean_.count())

Here I'm converting the datasets to .parquet format, and now it become less on disk

Before 153MB
After 66MB

And spark works faster with .parquet format, rather than csv

In [ ]:
fb_df_clean_.write.mode("overwrite").parquet("fb_df")
gg_df_clean_.write.mode("overwrite").parquet("gg_df")
wb_df_clean_.write.mode("overwrite").parquet("wb_df")